In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import json
import os

# Data prep
data_dir = "E:/Smart_Agro_Project/data/train"
img_size = 224
batch_size = 32

# Add this before using data_dir
import os

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Created missing directory: {data_dir}")
else:
    print(f"Directory exists: {data_dir}")

# Check if data_dir contains at least one class subdirectory with images
has_data = False
for entry in os.scandir(data_dir):
    if entry.is_dir():
        subdir = entry.path
        if any(fname.lower().endswith(('.png', '.jpg', '.jpeg')) for fname in os.listdir(subdir)):
            has_data = True
            break

if not has_data:
    raise ValueError(f"No image data found in '{data_dir}'. Please add at least one class subdirectory with images.")

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Save class map for prediction
class_dict = {v: k for k, v in train_data.class_indices.items()}
with open("disease_classes.json", "w") as f:
    json.dump(class_dict, f)

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=10)

# Save model
model.save("plant_disease_model.h5")


Directory exists: E:/Smart_Agro_Project/data/train
Found 3040 images belonging to 38 classes.
Found 760 images belonging to 38 classes.
Epoch 1/10
95/95 [==============================] - 101s 1s/step - loss: 3.6207 - accuracy: 0.1086 - val_loss: 2.5525 - val_accuracy: 0.3250
Epoch 2/10
95/95 [==============================] - 90s 950ms/step - loss: 2.1196 - accuracy: 0.4010 - val_loss: 1.8874 - val_accuracy: 0.4658
Epoch 3/10
95/95 [==============================] - 89s 931ms/step - loss: 1.2104 - accuracy: 0.6372 - val_loss: 1.5497 - val_accuracy: 0.5711
Epoch 4/10
95/95 [==============================] - 95s 997ms/step - loss: 0.7126 - accuracy: 0.7918 - val_loss: 1.5408 - val_accuracy: 0.5632
Epoch 5/10
32/95 [=========>....................] - ETA: 59s - loss: 0.4574 - accuracy: 0.8633 